In [1]:
import numpy as np
import gym
from IPython.display import clear_output, display
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import copy
import time
import matplotlib.pyplot as plt

In [2]:
env = gym.make('LunarLander-v2')
#Setting up the environment
seed_val = 456
env.seed(seed_val)
np.random.seed(seed_val)

In [3]:
class Network(nn.Module):
    
    def __init__(self, state_num , action_num, hidden_layer):
        
        super(Network, self).__init__()
        self.input_layer = nn.Linear(state_num, hidden_layer)
        self.h1_layer = nn.Linear(hidden_layer, hidden_layer)
        self.h2_layer = nn.Linear(hidden_layer, hidden_layer)
        self.output_layer = nn.Linear(hidden_layer, action_num)
        

    def forward(self, state):
        
        xh = F.relu(self.input_layer(state))
        hh1 = F.relu(self.h1_layer(xh))
        hh2 = F.tanh(self.h2_layer(hh1))
        state_action_values = self.output_layer(hh2)
        
        return state_action_values

In [4]:
class Q_Network(nn.Module):
    def __init__(self, state_dim , action_dim):
        super(Q_Network, self).__init__()
        self.x_layer = nn.Linear(state_dim, 150)
        self.h_layer = nn.Linear(150, 120)
        self.y_layer = nn.Linear(120, action_dim)
        print(self.x_layer)

    def forward(self, state):
        xh = F.relu(self.x_layer(state))
        hh = F.relu(self.h_layer(xh))
        state_action_values = self.y_layer(hh)
        return state_action_values

In [5]:
class DQNAgent(object):
    def __init__(self, state_dim, action_dim):
        self.qnet = Q_Network(state_dim, action_dim)
        self.qnet_optim = torch.optim.Adam(self.qnet.parameters(), lr=0.001)
        self.discount_factor = 0.99
        self.MSELoss_function = nn.MSELoss()
        self.replay_buffer = ReplayBuffer()
        pass
    
    def epsilon_greedy_action(self, state, epsilon):
        if np.random.uniform(0, 1) < epsilon:
                return env.action_space.sample()  # choose random action
        else:
                network_output_to_numpy = self.qnet(state).data.numpy()
                return np.argmax(network_output_to_numpy)  # choose greedy action

    def update_Q_Network(self, state, next_state, action, reward, terminals):
        
        qsa = torch.gather(self.qnet(state), dim=1, index=action.long())
        qsa_next_action = self.qnet(next_state)
        qsa_next_action,_ = torch.max(qsa_next_action, dim=1, keepdim=True)
        not_terminals = 1 - terminals
        qsa_next_target = reward + not_terminals * self.discount_factor * qsa_next_action
        q_network_loss = self.MSELoss_function(qsa, qsa_next_target.detach())
        self.qnet_optim.zero_grad()
        q_network_loss.backward()
        self.qnet_optim.step()
        
    
    def update_Sarsa_Network(self, state, next_state, action, next_action, reward, terminals):

        qsa = torch.gather(self.qnet(state), dim=1, index=action.long())

        qsa_next_action = torch.gather(self.qnet(next_state), dim=1, index=next_action.long())

        not_terminals = 1 - terminals

        qsa_next_target = reward + not_terminals * (self.discount_factor * qsa_next_action)

        q_network_loss = self.MSELoss_function(qsa, qsa_next_target.detach())
        self.qnet_optim.zero_grad()
        q_network_loss.backward()
        self.qnet_optim.step()
# FIXME
    def update_Bpessimistic_Network(self, state, next_state, action, reward, terminals):
        qsa = torch.gather(self.qnet(state), dim=1, index=action.long())
        
        qsa_next_action = self.qnet(next_state)
        qsa_best_next_action,_ = torch.max(qsa_next_action, dim=1, keepdim=True)
        qsa_worst_next_action,_ = torch.min(qsa_next_action, dim=1, keepdim=True)
        

        
        not_terminals = 1 - terminals
        qsa_next_target = reward + not_terminals * self.discount_factor * ((1-self.beta) * qsa_best_next_action + self.beta * qsa_worst_next_action)
        q_network_loss = self.MSELoss_function(qsa, qsa_next_target.detach())
        self.qnet_optim.zero_grad()
        q_network_loss.backward()
        self.qnet_optim.step()     
        
    def update(self, update_rate):
        for i in range(update_rate):
            states, next_states, actions, rewards, terminals = self.replay_buffer.sample_minibatch(64)
            states = torch.Tensor(states)
            next_states = torch.Tensor(next_states)
            actions = torch.Tensor(actions)
            rewards = torch.Tensor(rewards)
            terminals = torch.Tensor(terminals)
            self.update_Q_Network(states, next_states, actions, rewards, terminals)
            
    def update_s(self, update_rate):
        for i in range(update_rate):
            states, next_states, actions, next_actions, rewards, terminals = self.replay_buffer.sample_minibatch_sarsa(64)
            states = torch.Tensor(states)
            next_states = torch.Tensor(next_states)
            actions = torch.Tensor(actions)
            next_actions = torch.Tensor(next_actions)
            rewards = torch.Tensor(rewards)
            terminals = torch.Tensor(terminals)
            self.update_Sarsa_Network(states, next_states, actions, next_actions, rewards, terminals)
    
    def best_move(self, state):
        
        return np.argmax(self.qnet(state).data.numpy())

In [6]:
class ReplayBuffer(object):
    def __init__(self):
        self.buffer = []
        self.buffer_s = []
        
    def add_to_buffer(self, data):
        #data must be of the form (state,next_state,action,reward,terminal)
        self.buffer.append(data)
        
    def add_to_buffer_sarsa(self, data):
        #data must be of the form (state,next_state,action,n_action,reward,terminal)
        self.buffer_s.append(data)

    def sample_minibatch(self,minibatch_length):
        states = []
        next_states = []
        actions = []
        rewards = []
        terminals = []
        for i in range(minibatch_length):
            random_int = np.random.randint(0, len(self.buffer)-1) 
            transition = self.buffer[random_int]
            states.append(transition[0])
            next_states.append(transition[1])
            actions.append(transition[2])
            rewards.append(transition[3])
            terminals.append(transition[4])
        return torch.Tensor(states), torch.Tensor(next_states), torch.Tensor(actions), torch.Tensor(rewards), torch.Tensor(terminals)

    def sample_minibatch_sarsa(self,minibatch_length):
        states = []
        next_states = []
        actions = []
        next_actions = []
        rewards = []
        terminals = []
        for i in range(minibatch_length):
            random_int = np.random.randint(0, len(self.buffer_s)-1) 
            transition = self.buffer_s[random_int]
            states.append(transition[0])
            next_states.append(transition[1])
            actions.append(transition[2])
            next_actions.append(transition[3])
            rewards.append(transition[4])
            terminals.append(transition[5])
        return torch.Tensor(states), torch.Tensor(next_states), torch.Tensor(actions), torch.Tensor(next_actions), torch.Tensor

In [7]:
def m_a(values, window=50):
    weight = np.repeat(1.0, window)/window
    smas = np.convolve(values,weight,'valid')
    return smas

In [8]:
action_dim = env.action_space.n
state_dim = env.observation_space.shape[0]

# Deep Q-learning

In [9]:
agent = DQNAgent(state_dim, action_dim)
number_of_episodes = 600
max_time_steps = 2000
reward_list = []
epsilon = 1
finals = []


for episode in range(number_of_episodes):
    reward_sum = 0
    state = env.reset()

    for _ in range(max_time_steps):
        action = agent.epsilon_greedy_action(torch.from_numpy(state).float() , epsilon)
        next_state, reward, terminal, _ = env.step(action)
        
        reward_sum += reward
        agent.replay_buffer.add_to_buffer((state,next_state,[action],[reward],[terminal]))
        state = next_state
        if terminal:
            clear_output(wait=True)
            print('episode:', episode, 'sum_of_rewards_for_episode:', reward_sum)
            reward_list.append(reward_sum)
            finals.append(reward)
            break
            
    agent.update(128)
    
    if epsilon > 0.2:
        epsilon *= 0.995 #Epsilon decay
    
    if epsilon <= 0.2:
        epsilon = 0.2

episode: 456 sum_of_rewards_for_episode: 162.01339753507645


KeyboardInterrupt: 

In [ ]:
smas_ = m_a(reward_list)
fig, ax = plt.subplots(figsize=(10,5))

ax.plot(reward_list)
ax.plot(smas_)
ax.set_ylabel('Rewards')
ax.set_xlabel('Episodes')
ax.set_title('Deep Q learning (600 episodes training)')
plt.savefig('DQN600epma', transparent=True)

# Deep Sarsa

In [ ]:
agent = DQNAgent(state_dim, action_dim)
number_of_episodes = 600
max_time_steps = 2000
epsilon = 1
reward_list_sarsa = []
final_rewards = []

np.random.seed(0)
for episode in range(number_of_episodes):
    state = env.reset()
    
    reward_sum = 0
    
    action = agent.epsilon_greedy_action(torch.from_numpy(state).float(), epsilon)
    
    state_1, reward, terminal, _ = env.step(action)
    
    #Checks for early Finish
    if terminal:
        
        action_1 = agent.epsilon_greedy_action(torch.from_numpy(state_1).float(), epsilon)
        agent.replay_buffer.add_to_buffer_sarsa((state, state_1, [action], [action_1], [reward],[terminal]))
        
        reward_sum += reward
        
        final_rewards.append(reward)
        
        reward_list_sarsa.append(reward_sum)
        
        clear_output(wait=True)
        print('Early finish!', 'reward =', reward)
        print('episode:', episode, 'sum_of_rewards_for_episode:', reward_sum, 'final reward', \
                      reward, 'epsilon:', epsilon)
        
    #If not finished after first action - continue learning
    else:
        for i in range(max_time_steps):

            action_1 = agent.epsilon_greedy_action(torch.from_numpy(state_1).float(), epsilon)

            state_2, reward_1, terminal_1, _ = env.step(action_1)

            agent.replay_buffer.add_to_buffer_sarsa((state, state_1, [action], [action_1], [reward],[terminal]))

            reward_sum += reward

            state = state_1
            state_1 = state_2
            action = action_1
            reward = reward_1
            terminal = terminal_1

            if terminal:

                action_1 = agent.epsilon_greedy_action(torch.from_numpy(state_1).float(), epsilon)

                agent.replay_buffer.add_to_buffer_sarsa((state, state_1, [action], [action_1], [reward],[terminal]))

                reward_sum += reward
                
                final_rewards.append(reward)
                
                reward_list_sarsa.append(reward_sum)

                clear_output(wait=True)
                print('episode:', episode, 'sum_of_rewards_for_episode:', reward_sum, 'final reward', \
                      reward, 'epsilon:', epsilon)

                
                break
            
    agent.update_s(128) 
    
    if epsilon > 0.2:
        epsilon *= 0.995
    
    if epsilon <= 0.2:
        epsilon = 0.2

In [ ]:
smas = m_a(reward_list_sarsa)

fig, ax = plt.subplots(figsize=(10,5))

ax.plot(reward_list_sarsa)
ax.plot(smas)
ax.set_ylabel('Rewards')
ax.set_xlabel('Episodes')
ax.set_title('Deep SARSA learning (600 episodes training)')

# Deep B-pessimistic Q learning

In [ ]:
agent = DQNAgent(state_dim, action_dim)
number_of_episodes = 600
max_time_steps = 2000
reward_list = []
epsilon = 1
finals = []


for episode in range(number_of_episodes):
    reward_sum = 0
    state = env.reset()

    for _ in range(max_time_steps):
        action = agent.epsilon_greedy_action(torch.from_numpy(state).float() , epsilon)
        next_state, reward, terminal, _ = env.step(action)
        
        reward_sum += reward
        agent.replay_buffer.add_to_buffer((state,next_state,[action],[reward],[terminal]))
        state = next_state
        if terminal:
            clear_output(wait=True)
            print('episode:', episode, 'sum_of_rewards_for_episode:', reward_sum)
            reward_list.append(reward_sum)
            finals.append(reward)
            break
            
    agent.update_b(128)
    
    if epsilon > 0.2:
        epsilon *= 0.995 #Epsilon decay
    
    if epsilon <= 0.2:
        epsilon = 0.2

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.plot(reward_list)
ax.plot(smas_)
ax.set_ylabel('Rewards')
ax.set_xlabel('Episodes')
ax.set_title('Deep B-pessimistic Q learning (600 episodes training)')
